In [11]:
%matplotlib notebook
import numpy as np
import pandas as pd
import os
from dv import AedatFile
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import spikeFileIO

from sklearn.cluster import DBSCAN
import seaborn as sns

ModuleNotFoundError: No module named 'torch'

In [2]:
#파일 하나당 한 클래스 사용
class clustering:
    def __init__(self, frame_size = 40000):
        self.frame_size = frame_size
        self.events = None
        self.df = None
        self.start_time = 0
        
        
    def read_aedat_file(self, path):
        with AedatFile(path) as f:
            events = np.hstack([packet for packet in f['events'].numpy()])
            timestamps, x, y, polarities = events['timestamp'], events['x'], events['y'], events['polarity']
            self.events = events
            self.timestamp_zeroization()
            self.dflization()
            return events
        
    #read_aedat_file에서 호출
    def timestamp_zeroization(self):
        temp = self.events['timestamp'][0]
        for i in range(0,len(self.events['timestamp'])):
            self.events['timestamp'][i]-=temp
            
    #read_aedat_file에서 호출
    def dflization(self):
        self.df = pd.DataFrame(self.events)
            
    
    def cut_frame(self,start_time=0):
        end_condition = self. df['timestamp'] <= start_time+self.frame_size
        start_condition = self.df['timestamp'] >= start_time
        #print('end_condition : {}'.format(end_condition))
        #print('start_condition : {}'.format(start_condition))
        target_df = self.df[end_condition & start_condition]
        a = target_df.reset_index(drop=True)
        a = self.time_nomalization(a)
        return a
    
    def time_nomalization(self,frame):
        temp=self.frame_size/150
        cpy=pd.DataFrame.copy(frame)
        for i in range(len(frame)):
            cpy['timestamp'][i] = frame['timestamp'][i]/temp
        return cpy
    
    def make_model(self,feature, eps=17,min_samples=100):
        model = DBSCAN(eps=eps,min_samples=min_samples)
        predict = pd.DataFrame(model.fit_predict(feature))
        predict.columns=['predict']
        # concatenate labels to df as a new column
        r = pd.concat([feature,predict],axis=1)
        return r
    
        
        #그리기
    
def draw(r):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(r['x'],r['y'],r['timestamp'],c=r['predict'],alpha=0.5)
    ax.view_init(270,270)
    ax.set_xlabel('Sepal lenth')
    ax.set_ylabel('Sepal width')
    ax.set_zlabel('Petal length')
    
    plt.show()
    return r
        
def save(r,i):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(r['x'],r['y'],r['timestamp'],c=r['predict'],alpha=0.5)
    ax.view_init(270,270)
    ax.set_xlabel('Sepal lenth')
    ax.set_ylabel('Sepal width')
    ax.set_zlabel('Petal length')
    plt.savefig("./save/d/test{}.png".format(i))

In [3]:
#함수화

#판다스 배열들의 리스트 만들기
def event_output(p1):
    cname = ['timestamp','x','y','polarity','_p1','_p2','predict']


    r_list = []

    #검출한거 개수만큼 미리 리스트 만들어놓기
    r_list_size = p1['predict'].max(axis=0)
    for j in range(r_list_size+1):
        newp = pd.DataFrame(columns=cname)
        r_list.append(newp)

    for i in p1.index:
        #print(p1.loc[i].values)
        s=p1.loc[i].values.tolist()
        if p1.loc[i,"predict"] == -1:
            continue
        r_list[p1.loc[i,"predict"]].loc[len(r_list[p1.loc[i,"predict"]])]=s

    #print(r_list)
    return r_list

In [9]:
def csv_output(num,r_list):
    #num은 출력될 파일이름(전체에서 파일 순서)
    #r_list는 출력할 df 리스트
    list_size=len(r_list)
    
    for i in range(0,list_size):
        print("num : "+str(num+i))
        df=r_list[i]
        
        p_xEvent = df['x']
        p_yEvent = df['y']
        p_pEvent = df['polarity']
        p_tEvent = df['timestamp']
        #print(p_xEvent)
        # print('yyyyyyy')
        xEvent = []
        yEvent = []
        pEvent = []
        tEvent = []
        
        xEvent = p_xEvent.tolist()
        yEvent = p_yEvent.tolist()
        pEvent = p_pEvent.tolist()
        tEvent = p_tEvent.tolist()
        
        inputSpikes = spikeFileIO.event(xEvent, yEvent, pEvent, tEvent).toSpikeTensor(
            torch.zeros((2, 346, 260, 3000)), samplingTime=0.1
        )
        anim = spikeFileIO.animTD(spikeFileIO.spikeArrayToEvent(inputSpikes.reshape((2, 346, 260, -1)).cpu().data.numpy()))
        draw(df)
        save(df,num+i)
        df.to_csv("./csv/"+str(num+i)+".csv")
        
    #반환값은 새로운 num
    return num+list_size

In [10]:
test2 = clustering()
test2.read_aedat_file('./predata/newdata.aedat4')

num = 1
for i in range(1,2):
    size=i*40000
    sg1 = test2.cut_frame(size)
    sg2 = test2.make_model(sg1)
    sg3 = event_output(sg2)
    num = csv_output(num,sg3)
    #save(sg2,i)

C:\Users\sslab\anaconda3\envs\visual\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


num : 1


NameError: name 'torch' is not defined